# Рекомендация тарифов

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

Описание данных:

Каждый объект в наборе данных — это информация о поведении одного пользователя за месяц. Известно:
- сalls — количество звонков,
- minutes — суммарная длительность звонков в минутах,
- messages — количество sms-сообщений,
- mb_used — израсходованный интернет-трафик в Мб,
- is_ultra — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).

## Откройте и изучите файл

In [1]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score 
from tqdm import tqdm
from sklearn.dummy import DummyClassifier

In [2]:
df = pd.read_csv("/datasets/users_behavior.csv")

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


In [4]:
df.duplicated().sum()

0

In [5]:
df.head()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


### Вывод:
Пропусков и дубликатов не обнаруженно

## Разбейте данные на выборки

In [6]:
features = df.drop(["is_ultra"],axis=1)
target = df["is_ultra"]

In [7]:
features_train, features_valid, target_train, target_valid = train_test_split(features,
                                                                              target,
                                                                              test_size=0.4,
                                                                              random_state=12345
                                                                             )

In [8]:
features_valid, features_test ,target_valid, target_test = train_test_split(features_valid,
                                                                           target_valid,
                                                                           test_size=0.5,
                                                                           random_state=12345
                                                                           )

In [9]:
print(f'Размер исходного датафрейма: {df.shape}')

Размер исходного датафрейма: (3214, 5)


In [10]:
print(f'Размер признака обучающей выборке: {features_train.shape}')
print(f'Размер целевого признака обучающей выборке: {target_train.shape[0]}')

Размер признака обучающей выборке: (1928, 4)
Размер целевого признака обучающей выборке: 1928


In [37]:
print(f'Размер признака валидационной выборки: {features_valid.shape}')
print(f'Размер целевого признака валидационной выборки: {target_valid.shape[0]}')

Размер признака валидационной выборки: (643, 4)
Размер целевого признака валидационной выборки: 643


In [12]:
print(f'Размер признака тестовой выборки: {features_test.shape}')
print(f'Размер целевого признака тестовой выборки: {target_test.shape[0]}')

Размер признака тестовой выборки: (643, 4)
Размер целевого признака тестовой выборки: 643


## Исследуйте модели

### Дерево решений

In [13]:
best_model_tree = None
result_tree_best = 0
depth_tree = 0
for depth in tqdm(range(1,51)):
    model_tree = DecisionTreeClassifier(random_state=12345,max_depth=depth)
    model_tree.fit(features_train,target_train)  # обучение модели на тренировочной выборке
    predictions_tree_valid = model_tree.predict(features_valid) # получение предсказания модели на валидационной выборке
    result_tree = accuracy_score(target_valid,predictions_tree_valid) # рассчет accurace
    if result_tree > result_tree_best:
        best_model_tree = model_tree
        result_tree_best = result_tree
        depth_tree = depth
print("Информация о модели:")
print("Глубина дерева:",depth_tree)
print()
print("Accuracy")
print("Accuracy наилучшей модели на валидационной выборке:", result_tree_best)

100%|██████████| 50/50 [00:00<00:00, 61.10it/s]

Информация о модели:
Глубина дерева: 3

Accuracy
Accuracy наилучшей модели на валидационной выборке: 0.7853810264385692


#### Вывод:
Наибольший Accuracy в дереве решений составляет 0.79 при гиперпараметре max_depth (глубины дерева) равным 3

### Cлучайный лес

In [14]:
best_model_forest = None
result_forest_best = 0
depth_forest = 0
est_forest = 0
for est in tqdm(range(1,51)):
    for depth in range(1,51):
        model_forest = RandomForestClassifier(random_state=12345,max_depth=depth, n_estimators=est)
        model_forest.fit(features_train,target_train)  # обучение модели на тренировочной выборке
        predictions_forest_valid = model_forest.predict(features_valid) # получение предсказания модели на валидационной выборке
        result_forest = accuracy_score(target_valid,predictions_forest_valid) # рассчет accurace
        if result_forest > result_forest_best:
            best_model_forest = model_forest
            result_forest_best = result_forest
            depth_tree = depth
            est_forest = est
print("Информация о модели:")
print("Количесвто деревьев:",est_forest)
print("Глубина дерева:",depth_tree)
print()
print("Accuracy")
print("Accuracy наилучшей модели на валидационной выборке:", result_forest_best)

100%|██████████| 50/50 [05:53<00:00,  7.07s/it]

Информация о модели:
Количесвто деревьев: 40
Глубина дерева: 8

Accuracy
Accuracy наилучшей модели на валидационной выборке: 0.8087091757387247


#### Вывод:
При рассмотре различных гиперпараметров (колиечества деревьев и глубина дерева) было выявлено, что наибольший Accuracy достигается при 40 деревьев с глубиной 8.

### Логистическая регрессия

In [15]:
model_logistic = LogisticRegression(random_state=12345)
model_logistic.fit(features_train,target_train)  # обучение модели на тренировочной выборке
predictions_linear_valid = model_logistic.predict(features_valid) # получение предсказания модели на валидационной выборке
result_linear = accuracy_score(target_valid,predictions_linear_valid) # рассчет accurace


print("Accuracy")
print("Accuracy наилучшей модели на валидационной выборке:", result_linear)

Accuracy
Accuracy наилучшей модели на валидационной выборке: 0.7107309486780715


#### Вывод:
В логистической регрессии получился самый наименьший Accuracy - 0.71

### Вывод:
- В ходе исследовании моделей было выявлено, что наибольшим Accuracy обладает модель - случайный лес (0.8). При этом количество одновременных деревьев в модели составляет 40 штук, с глубиной дерева 8.
- Самым наименьшим значением обладает логистическая регрессия (0.71)

## Проверьте модель на тестовой выборке

### Дерево решений

In [26]:
#prediction_tree_test = best_model_tree.predict(features_test)
#accuracy_tree_test = accuracy_score(target_test,prediction_tree_test)
#print("Accuracy:",accuracy_tree_test)

### Случайный лес

In [27]:
prediction_forest_test = best_model_forest.predict(features_test)
accuracy_forest_test = accuracy_score(target_test,prediction_forest_test)
print("Accuracy:",accuracy_forest_test)

Accuracy: 0.7962674961119751


In [33]:
features_train_new, features_test_new, target_train_new, target_test_new = train_test_split(features,
                                                                              target,
                                                                              test_size=0.2,
                                                                              random_state=12345
                                                                             )

In [34]:
features_train_new.shape

(2571, 4)

In [35]:
features_test_new.shape

(643, 4)

In [36]:
model_forest_new = RandomForestClassifier(random_state=12345,max_depth=8, n_estimators=40)
model_forest_new.fit(features_train_new,target_train_new)  # обучение модели на тренировочной выборке
predictions_forest_valid_new = model_forest_new.predict(features_test_new) # получение предсказания модели на валидационной выборке
result_forest_new = accuracy_score(target_test_new,predictions_forest_valid_new) # рассчет accurace
print("Accuracy на новой выборке:", result_forest_new)

Accuracy на новой выборке: 0.8009331259720062


### Логистическая регрессия

In [38]:
#prediction_linear_test = model_logistic.predict(features_test)
#accuracy_linear_test = accuracy_score(target_test,prediction_linear_test)
#print("Accuracy:",accuracy_linear_test)

## Проверка модели на адекватность

In [23]:
dummy = DummyClassifier(strategy='most_frequent').fit(features_train, target_train)
dummy_pred = dummy.predict(features_test)
print('Test score: ', accuracy_score(target_test, dummy_pred))

Test score:  0.6842923794712286


### Вывод:
По полученным результатам, точность исследованных моделей получилась выше, чем у фиктивного классификатора. Что говорит о том, что наша модель работает адекватно.

## Общий вывод:

- Был изучен исходный файл. В результате не были обнаружены пропуски и дубликаты;
- Данные были разбиты на 3 выборки:  обучающая, валидационная и тестовая в соотношении 60:20:20 соответственно;
- Были использованы 3 модели с различными гиперпараметрами: Дерево решений, случайный лес и логистическая регрессия. В результате наибольшая точность получилась у случайного леса равная - 0.8. Наихудшая точность у логистической регрессии - 0.71;
- Модели проверены на тестовой выборке, где наибольшая точность также у случайного леса - 0.79;